<a href="https://colab.research.google.com/github/ADGGi/CP2/blob/main/Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
#!pip install mxnet
#!pip install gluonnlp pandas tqdm
#!pip install sentencepiece
#!pip install transformers==3.0.2
#!pip install torch

In [144]:
#!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 사용
device = torch.device("cuda:0")

#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [7]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/CP2/steam_text.txt', delimiter = '\t', names = ['label', 'comments'], header= 0)

In [8]:
df = df[['comments', 'label']]

In [9]:
#모델학습 데이터셋 만들기

data_list = []
for c, l in zip(df['comments'], df['label']):
  data = []
  data.append(c)
  data.append(str(l))
  data_list.append(data)

In [145]:
#data_list

#Train Test Split

In [11]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_list, test_size=0.3, random_state=722)

#KoBERT 입력 데이터로 만들기

In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [13]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#Tokenizatioin + padding

In [14]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

trainset = BERTDataset(train, 0, 1, tok, max_len, True, False)
testset = BERTDataset(test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [15]:
trainset[0]

(array([   2, 1212, 5859, 1177, 5439, 3608, 5859, 1177, 5439, 2919, 7086,
        1212, 2612, 4928, 7792, 1562, 1174, 6916, 7018, 6999,  492,  492,
        3941, 7172, 7788, 1212, 5940,  517, 6982, 5561, 7268, 5561, 2064,
        3715, 2377, 5761, 4427, 2590, 6669, 5703,  517,    5,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(43, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [16]:
train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#KoBERT 학습모델만들기

In [17]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [18]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [19]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1094 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7284504175186157 train acc 0.484375
epoch 1 batch id 201 loss 0.5606423020362854 train acc 0.5736162935323383
epoch 1 batch id 401 loss 0.5216242074966431 train acc 0.6534055486284289
epoch 1 batch id 601 loss 0.5290462970733643 train acc 0.6874740016638935
epoch 1 batch id 801 loss 0.3883218467235565 train acc 0.7128199126092385
epoch 1 batch id 1001 loss 0.6018156409263611 train acc 0.7288804945054945
epoch 1 train acc 0.7339650633046793


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.8106787491115849


  0%|          | 0/1094 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.1975298821926117 train acc 0.953125
epoch 5 batch id 201 loss 0.19823089241981506 train acc 0.9378886815920398
epoch 5 batch id 401 loss 0.19817273318767548 train acc 0.940149625935162
epoch 5 batch id 601 loss 0.19653886556625366 train acc 0.9439475873544093
epoch 5 batch id 801 loss 0.10555603355169296 train acc 0.947663077403246
epoch 5 batch id 1001 loss 0.18668292462825775 train acc 0.9493319180819181
epoch 5 train acc 0.9491584294410518


  0%|          | 0/469 [00:00<?, ?it/s]

epoch 5 test acc 0.8141102523098791


#테스트

In [45]:
#def softmax(vals, idx) :
#  valscpu = vals.cpu().detach().squeeze(0)
#  a = 0
#  for i in valscpu:
#    a += np.exp(i)
#  return ((np.exp(valscpu[idx]))/a).item()*100

#def testModel(model, seq) :
#  cate = ["부정", "긍정"]
#  tmp = [seq]
#  transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
#  tokenized = transform(tmp)

#  model.eval()
#  result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
#  idx = result.argmax().cpu().item()
#  print("라벨링 결과 :", cate[idx])
#  print("신뢰도는 :", "{:.2f}".format(softmax(result,idx)))

In [101]:
#testModel(model, "갑자기 바보를 박는다구?")

라벨링 결과 : 긍정
신뢰도는 : 97.01


In [133]:
def test2(comment) :
  cmtlist=[]
  sent = []
  for c in comment :
    cmtlist.append( [c, 5]) #[댓글, 임의의 양의 정수]

  cmtData = pd.DataFrame(cmtlist, columns = [['comments', 'label']])
  cmtData = cmtData.values
  test_set = BERTDataset(cmtData, 0, 1, tok, max_len, True, False)
  test_input = torch.utils.data.DataLoader(test_set, batch_size = batch_size, num_workers=2)

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input) :
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valie_length = valid_length
    #예측 결과 리스트 : out
    out = model(token_ids, valid_length, segment_ids)

    for e in out :
      if e[0]>e[1] :
        value = 0
      else :
        value = 1
      sent.append(value)

  return sent

#팬심 대화데이터셋 라벨링

In [25]:
fancim = pd.read_csv('/content/drive/MyDrive/CP2/통합문서3.csv')

In [28]:
cmt = fancim[['comments']][1:10327]
cmt

,comments
1,사랑해요🖤
2,님이 참여했습니다.
3,님이 참여했습니다.
4,단톡 전용 인스타 대체일지... 자려고했는데 파들..
5,이 님이 참여했습니다.
...,...
10322,ㅎㅎ 오늘도 비가 주륵주륵~ 벼보리님 우산 잘 쓰고 다니셔요 !
10323,Operator오늘은 ~ 수요일 ~ 스팀겜 하는 날~ 일주일의 절반이 벌써 다가왔네...
10324,재밌는시간 보내셨겠네요 ㅎㅎ\n저는 학교갔는데 생각했던것보단 정겹고 좋네요 은근 ㅎ...
10325,보리쟝이무화과를 먹었으니 저는 복숭아를 먹겠읍니다 후후\n푹 쉬고 7시에 방송에서...


In [102]:
testlist = cmt['comments'].tolist()

In [103]:
testlist2 = list(map(str, testlist))

In [134]:
test_label_list = test2(testlist2)

In [135]:
#길이 확인
len(test_label_list), len(testlist2)

(10326, 10326)

In [139]:
df_label = pd.DataFrame({
    'comments' : testlist2,
    'label' : test_label_list
})

In [142]:
df_label

,comments,label
0,사랑해요🖤,1
1,님이 참여했습니다.,1
2,님이 참여했습니다.,1
3,단톡 전용 인스타 대체일지... 자려고했는데 파들..,0
4,이 님이 참여했습니다.,1
...,...,...
10321,ㅎㅎ 오늘도 비가 주륵주륵~ 벼보리님 우산 잘 쓰고 다니셔요 !,1
10322,Operator오늘은 ~ 수요일 ~ 스팀겜 하는 날~ 일주일의 절반이 벌써 다가왔네...,1
10323,재밌는시간 보내셨겠네요 ㅎㅎ\n저는 학교갔는데 생각했던것보단 정겹고 좋네요 은근 ㅎ...,1
10324,보리쟝이무화과를 먹었으니 저는 복숭아를 먹겠읍니다 후후\n푹 쉬고 7시에 방송에서...,1


In [141]:
df_label.to_csv('/content/drive/MyDrive/CP2/df_label.csv',  index = False)